In [4]:
import os
import time
import requests
from typing import List, Dict
from getpass import getpass

# Helper function to securely set environment variables if they are not already present
def _set_env(var: str):
    # Check if the environment variable is not already set
    if not os.environ.get(var): # If not, prompt the user to enter it securely
        os.environ[var] = getpass(f"Enter your{var}: ")
_set_env("MINERU_TOKEN")
API_BASE = "https://mineru.net/api/v4"
TOKEN = os.getenv("MINERU_TOKEN") or "你的_API_TOKEN"

HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}


In [10]:
def apply_upload_urls(
    file_paths: List[str],
    model_version: str = "vlm",
) -> Dict:
    """
    向 MinerU 申请本地文件上传 URL
    """
    url = f"{API_BASE}/file-urls/batch"

    files_payload = []
    for path in file_paths:
        files_payload.append({
            "name": os.path.basename(path),
            "data_id": os.path.basename(path)  # 你也可以自定义
        })

    payload = {
        "files": files_payload,
        "model_version": model_version
    }

    resp = requests.post(url, headers=HEADERS, json=payload)
    resp.raise_for_status()

    data = resp.json()
    if data["code"] != 0:
        raise RuntimeError(f"申请上传 URL 失败: {data}")

    return data["data"]  # 包含 batch_id + file_urls


In [11]:
def upload_files(file_paths: List[str], upload_urls: List[str]):
    """
    PUT 上传文件到 MinerU 返回的 URL
    """
    assert len(file_paths) == len(upload_urls)

    for path, url in zip(file_paths, upload_urls):
        with open(path, "rb") as f:
            r = requests.put(url, data=f)
            if r.status_code != 200:
                raise RuntimeError(f"文件上传失败: {path}")
        print(f"✅ 上传成功: {path}")


In [12]:
def poll_batch_result(batch_id: str, interval: int = 5):
    """
    轮询解析任务状态，直到全部完成
    """
    url = f"{API_BASE}/extract-results/batch/{batch_id}"

    while True:
        resp = requests.get(url, headers=HEADERS)
        resp.raise_for_status()
        data = resp.json()

        if data["code"] != 0:
            raise RuntimeError(f"查询失败: {data}")

        results = data["data"]["extract_result"]

        all_done = True
        for r in results:
            state = r["state"]
            fname = r["file_name"]

            if state in ("pending", "running", "waiting-file", "converting"):
                all_done = False
                print(f"⏳ {fname} 状态: {state}")
            elif state == "failed":
                print(f"❌ {fname} 失败: {r.get('err_msg')}")
            elif state == "done":
                print(f"✅ {fname} 解析完成")

        if all_done:
            return results

        time.sleep(interval)


In [13]:
def download_result_zip(results: List[Dict], save_dir="mineru_results"):
    os.makedirs(save_dir, exist_ok=True)

    for r in results:
        if r["state"] != "done":
            continue

        zip_url = r["full_zip_url"]
        fname = r["file_name"].rsplit(".", 1)[0] + ".zip"
        save_path = os.path.join(save_dir, fname)

        resp = requests.get(zip_url)
        resp.raise_for_status()

        with open(save_path, "wb") as f:
            f.write(resp.content)

        print(f"📦 已下载: {save_path}")


In [14]:
def parse_local_documents(file_paths: List[str], save_dir="mineru_results"):
    # 1. 申请上传 URL
    upload_info = apply_upload_urls(file_paths, model_version="vlm")

    batch_id = upload_info["batch_id"]
    upload_urls = upload_info["file_urls"]

    # 2. 上传文件
    upload_files(file_paths, upload_urls)

    # 3. 轮询解析结果
    results = poll_batch_result(batch_id)

    # 4. 下载结果
    download_result_zip(results, save_dir=save_dir)

In [15]:
if __name__ == "__main__":
    file_path = r"C:\Users\22959\Desktop\数据清洗数据\未来乡村\CityDO数字乡村操作系统方案（初稿-0227）.doc"
    desktop = r"C:\Users\22959\Desktop"

    parse_local_documents([file_path], save_dir=desktop)


✅ 上传成功: C:\Users\22959\Desktop\数据清洗数据\未来乡村\CityDO数字乡村操作系统方案（初稿-0227）.doc
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: pending
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: pending
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: pending
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: pending
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: pending
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: pending
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: running
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: running
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: running
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: running
⏳ CityDO数字乡村操作系统方案（初稿-0227）.doc 状态: running
✅ CityDO数字乡村操作系统方案（初稿-0227）.doc 解析完成
📦 已下载: C:\Users\22959\Desktop\CityDO数字乡村操作系统方案（初稿-0227）.zip


In [16]:
from pathlib import Path
from typing import List

SUPPORTED_EXTENSIONS = {
    ".pdf",
    ".doc", ".docx",
    ".xls", ".xlsx",
    ".ppt", ".pptx"
}

def collect_parsable_files(folder_path: str) -> List[str]:
    """
    递归扫描目录，返回所有 MinerU 可解析文件的完整路径
    """
    folder = Path(folder_path)

    if not folder.exists():
        raise FileNotFoundError(f"路径不存在: {folder_path}")

    if not folder.is_dir():
        raise NotADirectoryError(f"不是文件夹: {folder_path}")

    files = []
    for path in folder.rglob("*"):
        if (
            path.is_file()
            and path.suffix.lower() in SUPPORTED_EXTENSIONS
            and not path.name.startswith("~$")   # Office 临时文件
        ):
            files.append(str(path))

    return files


In [ ]:
if __name__ == "__main__":
    root_dir = r"C:\Users\22959\Desktop\数据清洗数据\未来乡村"
    desktop = r"C:\Users\22959\Desktop"

    file_list = collect_parsable_files(root_dir)

    print(f"📄 共发现 {len(file_list)} 个可解析文件：")
    for f in file_list:
        print(f" - {f}")

    parse_local_documents(file_list, save_dir=desktop)
